In [5]:
import pandas as pd 
import json
import numpy as np 
import matplotlib.pyplot as plt 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
swords = set(stopwords.words('english'))

In [16]:
df = pd.read_json("./data/data71.json")
df1 = pd.read_json("./data/data70.json")

df = pd.concat([df, df1])
df.tail()

,name,message,timestamp
39924,Maplestori,Next time order 鱼柳包，then say 给我鸡胸肉 = no bone a...,2020-03-08 10:03:59
39925,is_10yrs,You want filet-o-fish.... boneless?,2020-03-08 09:59:43
39926,zesponk,are we even sure its fish bones?,2020-03-08 00:53:08
39927,prime5119,鱼胸肉,2020-03-08 10:06:36
39928,7_92x57_mm_Mauser,Could be plastic bones,2020-03-08 00:55:01


In [108]:
universities = [
    "Singapore Management University|smu",
    "nanyang technological university|ntu",
    "National University of Singapore|nus",
    "Singapore University of Technology and Design|sutd"
]

infoSys = "information systems|it|information technology|data analytics|computer science|com|computer|info systems|info sys"
simplifiedInfoSys = "computer|com|infosys|information systems|analytics"
simplifiedJC = "rank|a-level|a level|"

polyjc = []
with open("./data/polyjc.txt", "r") as f:
    for line in f:
        polyjc.append(line.strip()) 

def getScore(sentence): # return dict, e.g. --> {'neg': 0.0, 'neu': 0.326, 'pos': 0.674, 'compound': 0.7351}
    analyser = SentimentIntensityAnalyzer()
    return analyser.polarity_scores(sentence)

def writeFile(filepath, header, uni, startYear, endYear, queryRegexArray, domain=""):
    df = pd.read_json("./data/data71.json")
    # df1 = pd.read_json("./data/data70.json")
    # df = pd.concat([df, df1])

    f = open(filepath, "w+")
    f.write(header)
    for year in range(startYear, endYear + 1):
        dfYear = df[df["timestamp"].str.contains(str(year))]
        yearStr = str(year)
        for term in queryRegexArray:
            if domain != "" :
                term += "|" + domain
            listOfComments = dfYear[ dfYear["message"].str.contains(term, case=False) & dfYear["message"].str.contains(uni, case=False)]["message"].values.tolist()
            score = 0
            numOfComments = len(listOfComments) 
            for comment in listOfComments:
                score += getScore(comment)["compound"]
            overallScore = 0
            if numOfComments != 0:
                overallScore = score / numOfComments
            yearStr += "," + str(overallScore)
        f.write(yearStr)
        f.write("\n")
    f.close()

def printWeightiestSentences(filepath, header, uni, startYear, endYear, queryRegexArray, numOfSentences, domain=""):
    df = pd.read_json("./data/data70.json")
    df1 = pd.read_json("./data/data71.json")
    df = pd.concat([df, df1])
    print (df.info())
    f = open(filepath, "w+")
    f.write(header)

    for year in range(startYear, endYear + 1):
        dfYear = df[df["timestamp"].str.contains(str(year), na=False)]
        for term in queryRegexArray:
            if domain != "":
                term = term + "|" + domain
            listOfComments = dfYear[ dfYear["message"].str.contains(term, case=False) & dfYear["message"].str.contains(uni, case=False) ]["message"].values.tolist()

            vectorizer = TfidfVectorizer(stop_words=swords)

            try :
                X = vectorizer.fit_transform(listOfComments)

                feature_names = vectorizer.get_feature_names()

                vocab = vectorizer.vocabulary_

                unsorted_result = {}

                for i in range(len(list(X.toarray()))) :
                    row = list(list(X.toarray())[i])
                    unsorted_result[listOfComments[i]] = sum(row)
                
                result = pd.DataFrame()
                result["sentence"] = unsorted_result.keys()
                result["value"] = unsorted_result.values()

                print (result.head())

                df = result.sort_values(by=["value"], ascending=False)
                top10sentences = df.nlargest(numOfSentences, "value")["sentence"].tolist()
                top10values = df.nlargest(numOfSentences, "value")["value"].tolist()
                if len(top10sentences) >= numOfSentences:
                    for i in range(numOfSentences):
                        line = str(year) + "," + uni + "," + term + ",\"" + top10sentences[i] + "\"," + str(top10values[i]) + "\n"
                        f.write(line)
                        
                elif len(top10sentences) > 0:
                    for i in range(len(top10sentences)):
                        line = str(year) + "," + uni + "," + term + ",\"" + top10sentences[i] + "\"," + str(top10values[i]) + "\n"
                        f.write(line)

                else:
                    line = str(year) + "," + uni + "," + term + "," + ",\n"
                    f.write(line)

            except:
                line = str(year) + "," + uni + "," + term + "," + ",\n"
                f.write(line)


    f.close()

'''
The Positive, Negative and Neutral scores represent the proportion of text that falls in these categories. 
This means our sentence was rated as 67% Positive, 33% Neutral and 0% Negative. Hence all these should add up to 1.
The Compound score is a metric that calculates the sum of all the lexicon ratings 
which have been normalized between -1(most extreme negative) and +1 (most extreme positive).
'''


'\nThe Positive, Negative and Neutral scores represent the proportion of text that falls in these categories. \nThis means our sentence was rated as 67% Positive, 33% Neutral and 0% Negative. Hence all these should add up to 1.\nThe Compound score is a metric that calculates the sum of all the lexicon ratings \nwhich have been normalized between -1(most extreme negative) and +1 (most extreme positive).\n'

In [110]:
filepath = "./output/tfidf.csv"
header = "year,uni,search term,sentence,tfidf value\n"
uni = "Singapore Management University|smu"
start = 2014
end = 2014
arr = polyjc
num = 10

printWeightiestSentences(filepath, header, uni, start, end, arr, num)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 82417 entries, 0 to 42487
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       82417 non-null  object
 1   message    82417 non-null  object
 2   timestamp  82417 non-null  object
dtypes: object(3)
memory usage: 2.5+ MB
None
                                            sentence     value
0  I took Biz Finance BU8201 The killer part is t...  5.503282
1  wargoku wrote:  I took Biz Finance BU8201 The ...  6.667480
2  The original module in NBS (AB1201 - Financial...  6.534709
3  heenny wrote:  Appeal failed -.- Surprisingly,...  3.155357
4  Liao123 wrote:  Surprisingly, SMU is still pro...  2.995494
                                            sentence      value
0  Jackel_Yang wrote:  Wa Weaboo didi so kan chio...   7.344715
1  Darkzi0n wrote:  bcos final gpa or o lvl score...  12.260953
2  Irwin951 wrote:  Oh, are you from SMU? I have ...   5.871274
3  Zabiyaki wrote: